## Setup

In [1]:
import json
import re
import datetime
from collections import OrderedDict

import pandas as pd
import pandas.io.json as pj
from IPython.core.debugger import set_trace

import tbap.api as api
import tbap.server as server

In [2]:
key = "6Sx6a2gOyNWwNNTAd9UYBOpWp7MxaNrZgO4CqConwJwwDooKQV3DbZXvnk8TBL7A"
username = "stacy.k.irwin@gmail.com"
sn = api.Session(username, key)
snj = api.Session(username, key, data_format="json")

# Display floating point numbers in fixed format within dataframes.
pd.options.display.float_format = '{:f}'.format

### get_matches()

In [3]:
sn.time_zone = "America/Chicago"
matches = api.get_matches(sn, "2017tur")

In [9]:
matches.head()

,team_key,alliance,surrogate,score,disqualified,actual_time,comp_level,event_key,key,match_number,...,teleopFuelHigh,teleopFuelLow,teleopFuelPoints,teleopPoints,teleopRotorPoints,teleopTakeoffPoints,totalPoints,touchpadFar,touchpadMiddle,touchpadNear
0,frc2046,blue,False,503,False,2017-04-22 12:18:21-05:00,f,2017tur,2017tur_f1m1,1,...,63,0,21,391,120,150,503,ReadyForTakeoff,ReadyForTakeoff,ReadyForTakeoff
1,frc1318,blue,False,503,False,2017-04-22 12:18:21-05:00,f,2017tur,2017tur_f1m1,1,...,63,0,21,391,120,150,503,ReadyForTakeoff,ReadyForTakeoff,ReadyForTakeoff
2,frc1595,blue,False,503,False,2017-04-22 12:18:21-05:00,f,2017tur,2017tur_f1m1,1,...,63,0,21,391,120,150,503,ReadyForTakeoff,ReadyForTakeoff,ReadyForTakeoff
3,frc1296,red,False,271,False,2017-04-22 12:18:21-05:00,f,2017tur,2017tur_f1m1,1,...,17,0,5,185,80,100,271,ReadyForTakeoff,ReadyForTakeoff,None
4,frc2473,red,False,271,False,2017-04-22 12:18:21-05:00,f,2017tur,2017tur_f1m1,1,...,17,0,5,185,80,100,271,ReadyForTakeoff,ReadyForTakeoff,None


In [10]:
matches.shape

(770, 49)

In [11]:
matches = api.get_matches(sn, "2017tur", response="simple")

In [29]:
matches.time[0].utcoffset().seconds

68400

In [13]:
matches.shape

(770, 14)

In [ ]:
matches = api.get_matches(sn, "2017tur", response="keys")

In [ ]:
matches.head()

In [ ]:
matches = api.get_matches(sn, "2017tur", "frc1318")

In [ ]:
matches.head()

In [ ]:
matches = api.get_matches(sn, "2017tur", "frc1318", response="simple")

In [ ]:
matches[50:55]

In [ ]:
matches.shape

In [ ]:
matches = api.get_matches(sn, "2017tur", "frc1318", response="keys")

In [ ]:
matches.head()

In [ ]:
matches = api.get_matches(sn, team="frc1318", year=2017)

In [ ]:
matches.query("score == -1")

In [ ]:
matches.tail(6)

In [ ]:
matches.shape

In [ ]:
matches = api.get_matches(sn, team="frc1318", year=2017, response="simple")

In [ ]:
matches.head()

In [ ]:
matches = api.get_matches(sn, team="frc1318", year=2017, response="keys")

In [ ]:
matches.head()

In [ ]:
matches = api.get_matches(sn, match="2017wasno_qm79")

In [ ]:
matches

In [ ]:
matches.shape

In [ ]:
matches = api.get_matches(sn, match="2017wasno_qm79", response="simple")

In [ ]:
matches

In [ ]:
import pytz

In [ ]:
type(pytz.timezone("America/Chicago")).__bases__

In [ ]:
datetime.datetime.now(dateutil.tz.tzlocal()).tzname()

In [ ]:
matches.time.dt.tz_localize("UTC").dt.tz_convert("America/Sitka")

In [4]:
import pytz
import dateutil
import datetime
import dateutil.tz

In [9]:
tz = pytz.timezone("America/Los_Angeles")
str(tz)

'America/Los_Angeles'

In [ ]:
tm.tzname()